In [19]:
import pandas as pd
import numpy as np
import os
import glob
import math
from itertools import repeat
from collections import Counter
import bioframe
from collections import Counter
from pylab import rcParams
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import ScalarFormatter, AutoMinorLocator
import glob

In [17]:
bedgraph_extensions = ['.bedgraph', '.bedGraph', '.BedGraph', '.bg']
bigwig_extensions = ['.bigwig', '.bigWig', '.BigWig', '.bw']

In [2]:
def numeration(size, interaction_type = 'tad'):
    if size%2 == 0:
        dist = np.arange(size//2)
        dist = np.concatenate([dist, dist[::-1]])
    else:
        dist = np.arange((size-1)//2)
        dist = np.concatenate([dist, [size//2], dist[::-1]])
        
    if interaction_type == 'intertad':
        if len(dist) > 1: dist = (dist + 1) * np.nan
        else: dist = (dist + 1) * (-1)
    return dist


def columns(first_col, second_col, start, end, resolution, interaction_type = 'tad'):
    end  = resolution*math.ceil(end/resolution)
    size = (end-start)/resolution
    col1 = np.arange(start, end, resolution)
    
    if interaction_type != 'skip':
        col2 = numeration(int(size), interaction_type).astype(str)
    else:
        col2 = np.repeat(np.nan, math.ceil(size))

    first_col = np.append(first_col, col1)
    second_col = np.append(second_col, col2)
    return first_col, second_col.astype(str)


def get_distances(data, length, chromosome, resolution):
    first_col = np.array([])
    second_col = np.array([])
    
    bp = 0
    start_bin = int(data[0][1])
    if start_bin != 0:
        first_col, second_col = columns(first_col, second_col, bp, start_bin, resolution, interaction_type = 'skip')
    bp = start_bin
    
    for row in data:
        start_bin, end_bin = row[1], row[2]
        if bp != start_bin:
            first_col, second_col = columns(first_col, second_col, bp, start_bin, resolution, interaction_type = 'intertad')
        first_col, second_col = columns(first_col, second_col, start_bin, end_bin, resolution, interaction_type = 'tad')
        bp = end_bin
    
    
    if (length > bp):
        start, end = bp, length
        first_col, second_col = columns(first_col, second_col, start, end, resolution, interaction_type = 'skip')
    
    labels = np.repeat([chromosome], len(first_col))
    out = np.vstack((labels, first_col.astype(int), second_col))
    return np.transpose(out)


def nan_array_comparison(func, arr, thresh):
    # https://stackoverflow.com/questions/47340000/how-to-get-rid-of-runtimewarning-invalid-value-encountered-in-greater
    # by Divakar
    bool_arr = ~np.isnan(arr)
    bool_arr[bool_arr] = func(arr[bool_arr], thresh)
    return bool_arr


def equal_sizes(data, chromsize):
    data_size = dict(Counter(data.Chr.values))
    return data_size == chromsize


def check_prefix(vector):
    return ["chr" in chrm for chrm in vector]


def check_chrnames(labels_config, labels):
    if np.any(check_prefix(labels)) and not np.all(check_prefix(labels_config)):
        labels_config = ['chr'+ chrm for chrm in labels_config]
    elif not np.any(check_prefix(labels)) and np.all(check_prefix(labels_config)):   
        labels_config = [chrm[3:] for chrm in labels_config]
        
    if set(labels_config).issubset(set(labels)):
        chrnames = labels_config
    elif len(set(labels).intersection(labels_config)) > 0:
        chrnames = np.array(set(labels).intersection(labels_config))
    else:
        print('ERROR: Specified chromosomes are not found! Please set correct chromosome names in the configuration file.')
        sys.exit(1)
            
    return chrnames


def bin_calculation(chrdata, length, chrname, resolution):
    df = pd.DataFrame([])
    arr = []
    i = 0
    start_value = 0
    #print(chrdata.head(10))
    while i < length*resolution:
        d = chrdata.loc[(chrdata.Start < i+resolution) & (chrdata.Start >= i)]
        end_values = d.End.values
        if len(end_values) > 0:
            last_interval = end_values[-1] - (i+resolution)
            nbins = (end_values[-1] - d.Start.values[0])//resolution
            if nbins > 1:
                last_interval = last_interval%resolution
                new_bin = d.Score.values[-1]
            else:
                end_values[-1] -= last_interval
                interval_len = end_values - d.Start.values
                binvals = d.Score.values*interval_len/resolution
                new_bin = np.nansum(np.append(binvals, start_value))
                nbins = 1
            start_value = d.Score.values[-1]*last_interval/resolution
        
        else:
            new_bin = np.nan
            if i < chrdata.Start.values[0]:
                nbins = (chrdata.Start.values[0] - i)//resolution
            else:
                nbins = (length*resolution - i)//resolution
        
        for bin_idx in range(nbins):
            arr.append([chrname, str(i+bin_idx*resolution), str(i+resolution*(1+bin_idx)), str(new_bin)])
        
        i += resolution*nbins
    
    arr = np.reshape(arr, (i//resolution, 4))
    
    return arr


def binarize_data(data, chromsize, resolution):
    if equal_sizes(data, chromsize) != True:
        df = pd.DataFrame([])
        sizes = np.fromiter(chromsize.values(), dtype = int)
        for chrname, length in zip(np.unique(data.Chr.values), sizes):
            #print(chrname)
            chr_data = data.loc[data.Chr == chrname]
            arr = bin_calculation(chr_data, length, chrname, resolution)
            #arr = []
            #for i in np.arange(0, resolution*length, resolution):
            #    d = chr_data.loc[(chr_data.Start < i+resolution) & (chr_data.Start >= i)]
            #    arr.append([chrname, str(i), str(i+resolution), str(d.Score.mean(skipna=True))])

            #arr = np.reshape(arr, (length, 4))
            df = pd.concat([df, pd.DataFrame(arr)])
        data = df
        data.columns = ['Chr', 'Start', 'End', 'Score']
        data.replace(['inf', '-inf'], 'nan', inplace=True)
    
    return data


def blacklist_substraction(signal_data, bklst):
    keys = list(bklst.columns.values)
    i1 = signal_data.set_index(keys).index
    i2 = bklst.set_index(keys).index
    signal_data.loc[i1.isin(i2), 'Score'] = np.nan
    return signal_data


def get_bigwig_file(self, blacklist_regions):
    import pyBigWig
    bw = pyBigWig.open(self.path)
    if bw.isBigWig() == False:
        log.error('Incompatible format of ChIP-seq file!')
        sys.exit(1)

    df_chip = pd.DataFrame([])

    for ch in bw.chroms().keys():
        intervals = bw.intervals(ch)
        df_intervals = pd.DataFrame(intervals, columns = ['Start', 'End', 'Score'])
        chr_labels = list(repeat(ch, len(df_intervals.index)))
        df_chr = pd.concat([pd.DataFrame(chr_labels, columns = ['Chr']), df_intervals], axis = 1)
        if df_chip.empty:
            df_chip = df_chr
        else:
            df_chip = pd.concat([df_chip, df_chr])

        convert_dict = {'Chr': str, 'Start': int, 'End': int, 'Score': float}
        df_chip = df_chip.astype(convert_dict)
        

    if self.chrnames != ['']:
        labels = check_chrnames(self.chrnames, np.unique(df_chip.Chr))
        #print(labels)
        df_chip = df_chip.loc[df_chip['Chr'].isin(labels)]

    df_chip = binarize_data(df_chip, self.chromsize, self.resolution)

    return blacklist_substraction(df_chip, blacklist_regions)


def get_bedgraph(self, blacklist_regions):
    df_chip = pd.read_csv(self.path, sep = '\t', comment = 't', header = None, names = ['Chr', 'Start', 'End', 'Score'])

    if self.chrnames != ['']:
        labels = check_chrnames(self.chrnames, np.unique(df_chip.Chr))
        df_chip = df_chip.loc[df_chip['Chr'].isin(labels)]

    df_chip = df_chip.replace('NA', 'nan')
    df_chip.replace(['inf', '-inf'], 'nan', inplace=True)

    df_chip = binarize_data(df_chip, self.chromsize, self.resolution)
    convert_dict = {'Chr': str, 'Start': int, 'End': int, 'Score': float}
    df_chip = df_chip.astype(convert_dict)

    return blacklist_substraction(df_chip, blacklist_regions)


class ChipSeq:
    def __init__(self, path, set_chromosomes, chromsize, resolution):
        self.path = path
        self.extension = os.path.splitext(path)[1]
        self.chromsize = chromsize
        self.resolution = resolution
        print(self.path)

        if set_chromosomes != 'None':
            self.chrnames = set_chromosomes
        else:  
            self.chrnames = ['']

        accepted_extensions = bedgraph_extensions + bigwig_extensions
        if self.extension not in accepted_extensions:
            print('Incompatible format of ChIP-seq file!')
            sys.exit(1)

    def __call__(self, log2_chip, zscore_chip, blacklist_regions):
        if self.extension in bedgraph_extensions:
            df_chip = get_bedgraph(self, blacklist_regions)
        else:
            df_chip = get_bigwig_file(self, blacklist_regions)
        
        
        score = df_chip.Score.values.astype(float)
        bool_arr = nan_array_comparison(np.less, score, 1e-10)
        score[bool_arr] = np.nan
        df_chip.Score = score
        
        if log2_chip:
            score = df_chip.Score.values
            df_chip.Score = np.log2(score)
            df_chip = df_chip.replace(np.inf, np.nan)
        
        if zscore_chip:
            df_chip.Score = (df_chip.Score - df_chip.Score.mean()) / df_chip.Score.std(ddof=0)

        return df_chip

     
    
def get_stairs(index_data, chip_data, index_min = 0, index_max = 6, acetyl_min = -3, acetyl_max = 5, mammals = False):
    kb_list = np.arange(index_min, index_max, 1)
    gamma_range = index_data.keys()
    dict_amplitudes = {key: None for key in gamma_range}
    dict_stairs = {key: None for key in gamma_range}
    convert_chip = {'Chr': str, 'Start': int, 'End': int, 'Score': float}
    convert_dist = {'Chr': str, 'Bp': int, 'Index': str}
    
    for gamma in gamma_range:
        df_dist = pd.DataFrame(index_data[gamma][0], columns = ['Chr', 'Bp', 'Index'])
        chromosomes = index_data[gamma][1]
        
        df_dist = df_dist.loc[df_dist['Chr'].isin(chromosomes)]
        df_chip = chip_data.loc[chip_data['Chr'].isin(chromosomes)]
        
        df_dist = df_dist.astype(convert_dist)
        df_chip = df_chip.astype(convert_chip)
        
        df_dist = df_dist.sort_values(by = ["Chr", "Bp"])
        df_chip = df_chip.sort_values(by = ["Chr", "Start"])
        
        df_dist.index = np.arange(df_dist.shape[0])
        df_chip.index = np.arange(df_chip.shape[0])
        
        median_val = []
        interTAD = []
        TAD = []
            
        for idx in kb_list:
            row = df_dist.loc[df_dist.Index == str(idx)]
            if row.empty == False:
                bins = row.index.values
                d_chip = df_chip.loc[df_chip.index.isin(bins)]
                acetyl_val = d_chip['Score'].values
                acetyl_val = acetyl_val[~np.isnan(acetyl_val)]
                acetyl_val = acetyl_val[(acetyl_val > acetyl_min) & (acetyl_val < acetyl_max)]
                if len(acetyl_val) == 0:
                    median_val.append(np.nan)
                else:
                    median_val.append(np.median(acetyl_val))
                
                if idx < 1:
                    interTAD = np.append(interTAD, acetyl_val)
                if idx >= 1:
                    TAD = np.append(TAD, acetyl_val)
            else:
                median_val.append(np.nan)
        
        amplitude = np.median(interTAD) - np.median(TAD)
        dict_stairs[gamma] = np.array(median_val)
        dict_amplitudes[gamma] = amplitude
    
    return dict_stairs, dict_amplitudes


def stylize_axes(ax):
    ax.yaxis.major.formatter._useMathText = True
    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.yaxis.set_ticks_position('left')
    ax.xaxis.set_ticks_position('bottom')
    ax.tick_params(axis='both', which='major', labelsize=15)


def plotAmplitude(data, output_path = 'amplitude.png', dpi = 200):
    sns.set_palette(sns.color_palette('Set1'))
    samples = data.columns[1:]
    x_val = data.Gamma.values
    
    fig, ax = plt.subplots(figsize=(7, 5))
    for sample in samples:
        y_val = data[sample].values
        ax.plot(x_val, y_val, linewidth = 2.5, ls='--', color = 'steelblue', marker='o', label = sample)

    ax.set_xlabel('Window size', fontsize = 20)
    ax.set_ylabel('Amplitude', fontsize = 20)
    ax.legend(frameon=False, fontsize = 12)
    stylize_axes(ax)
    ax.grid(linestyle=':', linewidth='0.2', color='black')
    
    if output_path:
        path = os.path.dirname(output_path)
        if not os.path.exists(path) and path != '':
            os.makedirs(path, exist_ok=True)
        ax.figure.savefig(output_path, dpi=dpi, bbox_inches='tight')

    return ax


def plotStair(stair_df, index_min = -5, index_max = 5, output_path = 'stair.png', dpi = 200, path_to_stair_dataframe = None):
    sns.set_palette(sns.color_palette('Set1'))
    fig, ax = plt.subplots(figsize=(9, 7))
    for name in list(stair_df.keys()):
        x_val = np.arange(index_min, index_max, 1)
        y_val = stair_df[name].values
    
        if np.isnan(y_val).any() == True:
            idx = np.argwhere(np.isnan(y_val))
            idx = np.concatenate(idx)
            x_val = np.delete(x_val, idx)
            y_val = np.delete(y_val, idx)
    
        poly = np.polyfit(x_val, y_val, 3)
        poly_y = np.poly1d(poly)(x_val)
        ax.plot(x_val, y_val, linewidth = 2.5, color = 'steelblue')
        #ax.plot(x_val, poly_y, linewidth = 2.5, color = 'steelblue')

    ax.set_xlabel('Distance to TAD boundary, kb', fontsize = 20)
    ax.set_ylabel('Median z-score of acetylation values', fontsize = 20)
    ax.grid(linestyle=':', linewidth='0.2', color='black')
    
    stylize_axes(ax)
    ax.legend(frameon = False, fontsize = 12)

    if output_path:
        path = os.path.dirname(output_path)
        if not os.path.exists(path) and path != '':
            os.makedirs(path, exist_ok=True)
        ax.figure.savefig(output_path, dpi=dpi, bbox_inches='tight')

    return ax

In [18]:
resolution = 20000
manual_chr_selection = False
chr_to_take = ['chr1', 'chr2', 'chr3']
log2_chip = True
zscore_chip = True

## Neurons

In [10]:
ins = pd.read_csv(os.path.join(os.path.realpath('..'), 
                               '../../../../HiC_data/optimalTAD_mammals/Data/20K/Neurons/true_mouse_sampled_neurons_table_20K.txt'), sep="\t")
names = ins['chrom'].values
names = np.char.array(np.repeat('chr', len(names))) + np.char.array(names.astype(str))
ins['chrom'] = names

chr_names = np.unique(ins.chrom)

chr_names = [x for x in chr_names if "_" not in x ]
chr_names = [x for x in chr_names if "M" not in x ]
chr_names = [x for x in chr_names if "Y" not in x ]
chr_names = [x for x in chr_names if "X" not in x ]

if manual_chr_selection:
    chr_names = chr_to_take
    
chr_sizes = [max(ins[ins.chrom == x].end.values) for x in chr_names]

<ipython-input-10-cc6333796572>:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ins = pd.read_csv(os.path.join(os.path.realpath('..'),


In [11]:
list_of_sizes = [i for i in ins.columns if i.find('is_boundary') == 0]

for size in list_of_sizes:
    ins.loc[(ins[size] == True) & (ins[f"is_bad_bin"] == True), size] = False

In [12]:
blacklist = ins.loc[ins.is_bad_bin == True, ["chrom", "start", "end"]]

In [13]:
blacklist.columns = ['Chr', 'Start', 'End']

In [14]:
window_sizes = [i.split("_")[2] for i in ins.columns if i.find('is_boundary') == 0]

In [15]:
tad_files = dict()
#ins = ins[ins[f"is_bad_bin"] == False]
for window_size in window_sizes:
    ins_ws = ins[ins[f"is_boundary_{window_size}"] == False]
    tads = bioframe.merge(ins_ws)
    tads = tads[(tads["end"] - tads["start"]) <= 2000000].reset_index(drop=True)
    tads = tads[['chrom', 'start', 'end']]
    tads.columns = ['Chr', 'Start', 'End']
    if manual_chr_selection:
        tads = tads.loc[tads['Chr'].isin(chr_to_take)]
    tad_files[window_size] = tads

In [16]:
dict_md = {key: [] for key in tad_files.keys()}
df = pd.DataFrame()
d = tad_files['60000']
lbl_total = chr_names
a1= get_distances(d.loc[d['Chr'] == chr_names[0]].values, chr_sizes[0], chr_names[0], resolution)


dict_md = {key: [] for key in tad_files.keys()}
for window_size in tad_files.keys():
    df = pd.DataFrame()
    d = tad_files[window_size]
    lbl_total = chr_names
    for length, label in zip(chr_sizes, chr_names):
        markdown = get_distances(d.loc[d['Chr'] == label].values, length, label, resolution)
        df_chr = pd.DataFrame(markdown)
        df = pd.concat([df, df_chr])
    
    dict_md[window_size].append(df.values)
    dict_md[window_size].append(lbl_total)

In [20]:
ctype = 'ACC'

In [23]:
path_to_signals = os.path.join(os.path.realpath('..'), '../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/')

In [24]:
for chipseq_path in glob.glob(path_to_signals+ctype+"*bw"):
    ChipSeqLoader = ChipSeq(chipseq_path, chr_names, dict(Counter(dict_md['80000'][0][:,0])), resolution)
    chip_data = ChipSeqLoader(log2_chip, zscore_chip, blacklist)
    stairs, amplitudes = get_stairs(dict_md, chip_data, index_min = -1, index_max = 80, acetyl_min = -10, acetyl_max = 10, mammals = True)
    suffix = chipseq_path.split('/')[-1]
    df_sample = pd.DataFrame(amplitudes.items(), columns = ['Gamma', suffix.split('.')[0]])
    df_sample.Gamma = df_sample.Gamma.astype(int)
    df_sample = df_sample.sort_values(by=['Gamma'])
    stair_dict = {}
    stair_dict[suffix.split('.')[0]] = stairs['60000']
    stair_df = pd.DataFrame(stair_dict)
    suffix = chipseq_path.split('/')[-1]
    
    path_short = chipseq_path.split('/')[-1].split('.')[0]
    path_short = path_short.split('-')[2].split('_')[0]
    path_to_amplitudes = "../Output/Neurons/20K/"+path_short+"/Amplitudes_"+path_short+".csv"
    path_to_stairs = "../Output/Neurons/20K/"+path_short+"/Stairs_"+path_short+".csv"
    
    #Uncomment the lines below to save the output 
    #df_sample.to_csv(path_to_amplitudes, header = True, index=True)
    #stair_df.to_csv(path_to_stairs, header = True, index=True)

/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/ACC-NEU-H3K79ME3_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/ACC-NEU-MC_not20K_notdivided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/ACC-NEU-H3K27AC_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/ACC-NEU-H3K4ME1_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/ACC-NEU-CTCF_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/ACC-NEU-H3K4ME3_20K_divided_by_in

## Glia

In [26]:
ins = pd.read_csv(os.path.join(os.path.realpath('..'), 
                               '../../../../HiC_data/optimalTAD_mammals/Data/20K/Glia/true_mouse_glia_table_20K.txt'), sep="\t")
                               
names = ins['chrom'].values
names = np.char.array(np.repeat('chr', len(names))) + np.char.array(names.astype(str))
ins['chrom'] = names

chr_names = np.unique(ins.chrom)

chr_names = [x for x in chr_names if "_" not in x ]
chr_names = [x for x in chr_names if "M" not in x ]
chr_names = [x for x in chr_names if "Y" not in x ]
chr_names = [x for x in chr_names if "X" not in x ]

if manual_chr_selection:
    chr_names = chr_to_take
    
chr_sizes = [max(ins[ins.chrom == x].end.values) for x in chr_names]

<ipython-input-26-f34edf0cbc07>:1: DtypeWarning: Columns (0,3) have mixed types. Specify dtype option on import or set low_memory=False.
  ins = pd.read_csv(os.path.join(os.path.realpath('..'),


In [27]:
list_of_sizes = [i for i in ins.columns if i.find('is_boundary') == 0]

for size in list_of_sizes:
    ins.loc[(ins[size] == True) & (ins[f"is_bad_bin"] == True), size] = False

In [29]:
blacklist = ins.loc[ins.is_bad_bin == True, ["chrom", "start", "end"]]
blacklist.columns = ['Chr', 'Start', 'End']

In [30]:
window_sizes = [i.split("_")[2] for i in ins.columns if i.find('is_boundary') == 0]

In [31]:
tad_files = dict()
#ins = ins[ins[f"is_bad_bin"] == False]
for window_size in window_sizes:
    ins_ws = ins[ins[f"is_boundary_{window_size}"] == False]
    tads = bioframe.merge(ins_ws)
    tads = tads[(tads["end"] - tads["start"]) <= 2000000].reset_index(drop=True)
    tads = tads[['chrom', 'start', 'end']]
    tads.columns = ['Chr', 'Start', 'End']
    if manual_chr_selection:
        tads = tads.loc[tads['Chr'].isin(chr_to_take)]
    tad_files[window_size] = tads

In [32]:
dict_md = {key: [] for key in tad_files.keys()}
df = pd.DataFrame()
d = tad_files['60000']
lbl_total = chr_names
a1= get_distances(d.loc[d['Chr'] == chr_names[0]].values, chr_sizes[0], chr_names[0], resolution)


dict_md = {key: [] for key in tad_files.keys()}
for window_size in tad_files.keys():
    df = pd.DataFrame()
    d = tad_files[window_size]
    lbl_total = chr_names
    for length, label in zip(chr_sizes, chr_names):
        markdown = get_distances(d.loc[d['Chr'] == label].values, length, label, resolution)
        df_chr = pd.DataFrame(markdown)
        df = pd.concat([df, df_chr])
    
    dict_md[window_size].append(df.values)
    dict_md[window_size].append(lbl_total)

In [33]:
ctype = 'NON'

In [34]:
path_to_signals = os.path.join(os.path.realpath('..'), '../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/')

In [35]:
for chipseq_path in glob.glob(path_to_signals+ctype+"*bw"):
    ChipSeqLoader = ChipSeq(chipseq_path, chr_names, dict(Counter(dict_md['80000'][0][:,0])), resolution)
    chip_data = ChipSeqLoader(log2_chip, zscore_chip, blacklist)
    stairs, amplitudes = get_stairs(dict_md, chip_data, index_min = -1, index_max = 80, acetyl_min = -10, acetyl_max = 10, mammals = True)
    suffix = chipseq_path.split('/')[-1]
    df_sample = pd.DataFrame(amplitudes.items(), columns = ['Gamma', suffix.split('.')[0]])
    df_sample.Gamma = df_sample.Gamma.astype(int)
    df_sample = df_sample.sort_values(by=['Gamma'])
    stair_dict = {}
    stair_dict[suffix.split('.')[0]] = stairs['80000']
    stair_df = pd.DataFrame(stair_dict)
    suffix = chipseq_path.split('/')[-1]
    
    path_short = chipseq_path.split('/')[-1].split('.')[0]
    path_short = path_short.split('-')[2].split('_')[0]
    path_to_amplitudes = "../Output/Glia/20K/"+path_short+"/Amplitudes_"+path_short+".csv"
    path_to_stairs = "../Output/Glia/20K/"+path_short+"/Stairs_"+path_short+".csv"
    
    #Uncomment the lines below to save the output 
    #df_sample.to_csv(path_to_amplitudes, header = True, index=True)
    #stair_df.to_csv(path_to_stairs, header = True, index=True)

/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/NON-NEU-H3K4ME1_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/NON-NEU-H3K27AC_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/NON-NEU-H3K27ME3_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/NON-NEU-H3K79ME3_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/NON-NEU-H3K4ME3_20K_divided_by_input.bw
/Users/dmitriismirnov/Dropbox/Projects/optimalTAD/JupyterNotebooks/../../../../HiC_data/optimalTAD_mammals/Data/aggregation_new/NON-NEU-MC_not20K_notdivided_b